In [1]:
%matplotlib inline
import numpy as np

**作者**：Emmanuelle Gouillart, Gaël Varoquaux

这个部分解决用核心的科学模块NumPy和SciPy做基本的图像操作和处理。这个教程中涵盖的一些操作可能对于一些其他类型的多维度数据处理比对图像处理更加有用。特别是，子摸块[scipy.ndimage](http://docs.scipy.org/doc/scipy/reference/ndimage.html#module-scipy.ndimage)提供了在N维Numpy数组上操作的方法。

---
**也看一下: ** 对于更高级的图像处理和图像特有的程序，见专注于skimage模块教程[Scikit-image: 图像处理](http://www.scipy-lectures.org/packages/scikit-image/index.html#scikit-image)。

---